In [90]:

import wandb

import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import os
from glob import glob
import sys
import argparse



import torchaudio
import json
#import torchaudio.functional as F


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score




from tqdm import tqdm

# # SVD 문장 데이터에서 Feature 추출
# - mfcc

import pickle
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
import scipy
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.

In [91]:
# classifier
from xgboost import XGBClassifier
# import svm classifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [92]:
### paths
from Dataset.cv_spliter import cv_spliter 

In [93]:
#use pickle to load data which path is "../../voice_data/all_data_ver2/phrase_dict_ver2_all.pickle"
def load_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

phrase=load_data("../../voice_data/all_data_ver2/phrase_dict_ver2_all.pickle")
DataFrame=pd.read_excel("../../voice_data/all_data_ver2.xlsx")

In [94]:
random_state=1004
speaker_file_path = "../../voice_data/only_organics_healthy_available_ver2.xlsx" # 퓨전셋에 맞게 01.10 수정
speaker_file_path_abs = os.path.abspath(speaker_file_path)
data_probs=0

X_train_list, X_valid_list, X_test, Y_train_list, Y_valid_list, Y_test = cv_spliter(random_state, speaker_file_path_abs, data_probs=data_probs)

426
630
총 데이터수 :  1056
---
훈련 셋 :  844 Counter({'healthy': 504, 'pathology': 340})
테스트 셋 :  212 Counter({'healthy': 126, 'pathology': 86})
---
교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 272}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 272}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 272}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 403, 'pathology': 272}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 68} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 404, 'pathology': 272}
검증 레이블 데이터 분포 : 
 {'healthy': 100, 'pathology': 68} 

train. speaker to voice
valid. speaker to voice
test. speaker to voice

 fold0 
before dataset shape Counter({'healthy': 406, 'pathology': 364})
Resampled dataset shape Counter({'healthy': 406, 'pathology': 406})

 fold1 
before dataset shape Counter({'healthy': 407, 'pat

In [95]:
# write function which transforms list [[1,7,8],[1,2,3,4,5]] -> [["1-phrase.wav", "7-phrase.wav", "8-phrase.wav"], ["1-phrase.wav", "2-phrase.wav", "3-phrase.wav", "4-phrase.wav", "5-phrase.wav"]]
def list_to_path(list,dataset):
    path_list=[]
    for i in list:
        path_list.append([str(j)+"-"+dataset+".wav" for j in i])
    return path_list

# write function which transforms list [1,7,8] -> ["1-phrase.wav", "7-phrase.wav", "8-phrase.wav"]
def list_to_path2(list,dataset):
    path_list=[]
    for i in list:
        path_list.append(str(i)+"-"+dataset+".wav")
    return path_list

X_train_list=list_to_path(X_train_list,'phrase')
X_valid_list=list_to_path(X_valid_list,'phrase')
X_test=list_to_path2(X_test,'phrase')

In [96]:
def statistical_feature(feature_vec):
    mean = np.mean(feature_vec)
    std = np.std(feature_vec) 
    maxv = np.amax(feature_vec) 
    minv = np.amin(feature_vec) 
    median = np.median(feature_vec)
    skew = scipy.stats.skew(feature_vec)
    kurt = scipy.stats.kurtosis(feature_vec)
    q1 = np.quantile(feature_vec, 0.25)
    q3 = np.quantile(feature_vec, 0.75)
    mode = scipy.stats.mode(feature_vec)[0][0]
    iqr = scipy.stats.iqr(feature_vec)

    return [mean, std, maxv, minv, median, skew, kurt, q1, q3, mode, iqr]


def load_audio_dataset(audio_files,sr=16000):
    """
    Load audio dataset and extract features.
    """
    # Load audio files and extract features
    X = []
    y = []
    # Replace this with your own logic to load audio files and their corresponding labels

    # Loop through each audio file
    for audio_path in tqdm(audio_files):
        # Load audio file
        audio = phrase[audio_path]

        # Extract F0 using pitch detection (e.g., yin)
        f0 = librosa.yin(audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), frame_length=1024, hop_length=512)
        amplitude = librosa.feature.rms(y=audio, frame_length=1024, hop_length=512)
        # Extract Jitter
        jitter = (abs(f0[1:] - f0[:-1]).mean() / f0.mean()) * 100
        # Extract Shimmer

        shimmer = 20*abs(np.log10(amplitude[0][1:]/amplitude[0][:-1])).mean()
        # Extract HNR (Harmonic-to-Noise Ratio)
        hnr = librosa.effects.harmonic(audio)
        
        # Extract 13 MFCC (Mel-frequency cepstral coefficients)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)

        # Extract first and second derivatives of MFCC
        mfcc_delta = librosa.feature.delta(mfcc)
        mfcc_delta2 = librosa.feature.delta(mfcc, order=2)

        # Concatenate all features into a single feature vector
        features = np.concatenate([statistical_feature(f0), #[jitter], [shimmer], 
                                   statistical_feature(hnr),statistical_feature(mfcc.flatten()), statistical_feature(mfcc_delta.flatten()), statistical_feature(mfcc_delta2.flatten()) ], axis=0)

        # Append feature vector and label to X and y, respectively
        #print(features.shape)
        X.append(features)

    return np.array(X)

In [97]:
X_test_data = load_audio_dataset(X_test,16000)

for i in range(len(X_train_list)):
    X_train_list_data=load_audio_dataset(X_train_list[i],16000)
    X_valid_list_data=load_audio_dataset(X_valid_list[i],16000)
    

    # svm classifier fit X_train_list_data and Y_train_list[i]
    svm = SVC(kernel='linear', C=1, random_state=0)
    svm.fit(X_train_list_data, Y_train_list[i])
    # save svm checkpoint
    #with open('./checkpoint/svm_checkpoint_'+str(i)+'.pkl', 'wb') as f:
    #    pickle.dump(svm, f)
    # 
    # Test onto X_valid_list_data
    y_pred = svm.predict(X_valid_list_data)
    # Calculate accuracy score
    acc = accuracy_score(Y_valid_list[i], y_pred)
    # Calculate F1 score
    f1 = f1_score(Y_valid_list[i], y_pred, average='macro')
    # Calculate confusion matrix
    cm = confusion_matrix(Y_valid_list[i], y_pred)

    print("Fold valid: ", i)
    print("Accuracy: ", acc)
    print("F1 score: ", f1)
    print("Recall: ", cm[0][0]/(cm[0][0]+cm[0][1]))
    print("Specificity: ", cm[1][1]/(cm[1][0]+cm[1][1]))
    #UAR score. half of recall and specificity
    print("UAR score: ", (cm[0][0]/(cm[0][0]+cm[0][1])+cm[1][1]/(cm[1][0]+cm[1][1]))/2)
    print("Confusion matrix: \n", cm)
    print("--------------------")

    # Test onto X_test_data
    y_pred = svm.predict(X_test_data)
    # Calculate accuracy score
    acc = accuracy_score(Y_test, y_pred)
    # Calculate F1 score
    f1 = f1_score(Y_test, y_pred, average='macro')
    # Calculate confusion matrix
    cm = confusion_matrix(Y_test, y_pred)
    print("Fold test: ", i)
    print("Accuracy: ", acc)
    print("F1 score: ", f1)
    print("Recall: ", cm[0][0]/(cm[0][0]+cm[0][1]))
    print("Specificity: ", cm[1][1]/(cm[1][0]+cm[1][1]))
    #UAR score. half of recall and specificity
    print("UAR score: ", (cm[0][0]/(cm[0][0]+cm[0][1])+cm[1][1]/(cm[1][0]+cm[1][1]))/2)
    print("Confusion matrix: ", cm)    
    print("--------------------")





    
       


100%|██████████| 223/223 [00:13<00:00, 16.65it/s]


Fold valid:  0
Accuracy:  0.7623318385650224
F1 score:  0.7618529488806947
Recall:  0.7843137254901961
Specificity:  0.743801652892562
UAR score:  0.7640576891913791
Confusion matrix: 
 [[80 22]
 [31 90]]
--------------------
Fold test:  0
Accuracy:  0.7805907172995781
F1 score:  0.7791714449541284
Recall:  0.8095238095238095
Specificity:  0.7477477477477478
UAR score:  0.7786357786357787
Confusion matrix:  [[102  24]
 [ 28  83]]
--------------------


100%|██████████| 184/184 [00:10<00:00, 16.78it/s]


Fold valid:  1
Accuracy:  0.75
F1 score:  0.7440735365263667
Recall:  0.8217821782178217
Specificity:  0.6626506024096386
UAR score:  0.7422163903137302
Confusion matrix: 
 [[83 18]
 [28 55]]
--------------------
Fold test:  1
Accuracy:  0.7426160337552743
F1 score:  0.7378366125668692
Recall:  0.8253968253968254
Specificity:  0.6486486486486487
UAR score:  0.737022737022737
Confusion matrix:  [[104  22]
 [ 39  72]]
--------------------


100%|██████████| 202/202 [00:12<00:00, 16.51it/s]


Fold valid:  2
Accuracy:  0.7376237623762376
F1 score:  0.7374629098310405
Recall:  0.7128712871287128
Specificity:  0.7623762376237624
UAR score:  0.7376237623762376
Confusion matrix: 
 [[72 29]
 [24 77]]
--------------------
Fold test:  2
Accuracy:  0.7552742616033755
F1 score:  0.7540085898353616
Recall:  0.7777777777777778
Specificity:  0.7297297297297297
UAR score:  0.7537537537537538
Confusion matrix:  [[98 28]
 [30 81]]
--------------------


100%|██████████| 190/190 [00:11<00:00, 16.72it/s]


Fold valid:  3
Accuracy:  0.7631578947368421
F1 score:  0.7602288342353964
Recall:  0.8217821782178217
Specificity:  0.6966292134831461
UAR score:  0.7592056958504839
Confusion matrix: 
 [[83 18]
 [27 62]]
--------------------
Fold test:  3
Accuracy:  0.7468354430379747
F1 score:  0.7448320413436693
Recall:  0.7857142857142857
Specificity:  0.7027027027027027
UAR score:  0.7442084942084942
Confusion matrix:  [[99 27]
 [33 78]]
--------------------


100%|██████████| 194/194 [00:11<00:00, 16.28it/s]


Fold valid:  4
Accuracy:  0.8092783505154639
F1 score:  0.8092327318149201
Recall:  0.7766990291262136
Specificity:  0.8461538461538461
UAR score:  0.8114264376400299
Confusion matrix: 
 [[80 23]
 [14 77]]
--------------------
Fold test:  4
Accuracy:  0.759493670886076
F1 score:  0.7588755020080322
Recall:  0.7619047619047619
Specificity:  0.7567567567567568
UAR score:  0.7593307593307593
Confusion matrix:  [[96 30]
 [27 84]]
--------------------
